In [ ]:
import pandas as pd
import polars as pl
import tqdm
import numpy as np
from scipy.spatial.distance import cdist
import os
import plotly.express as px
import gc

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc = {'figure.figsize':(15,8)})

import warnings
warnings.filterwarnings('ignore')

# Questions to answer

1. Trend in delay avg daily delay times across months
2. Trend in avg delay times across days of week, Overall, by area,  by top ten routes
3. Trend in avg delay times by time of day , Overall, by area,  by top ten routes
4. weather effect
5. traffic effect????
6. area effect
7. Efficiency by area (maybe) see paper

# 1. Introduction

In [ ]:
fields =['Stop Number'
# ,'Route Number'	
,'Route Name'	
,'Route Destination'	
,'Day Type'	
,'Scheduled Time'
,'Deviation'	
,'date_key'	
,'windspeed'	
,'precip'		
,'preciptype'					
,'Total'	
,'id'	
,'name']

dtype = {'Stop Number':pl.Int64
# ,'Route Number':pl.Object
,'Route Name':pl.Utf8
,'Route Destination':pl.Utf8
,'Day Type':pl.Utf8	
,'Scheduled Time':pl.Datetime
,'Deviation':pl.Int64
,'date_key':pl.Datetime	
,'windspeed':pl.Float64
,'precip':pl.Float64
,'preciptype':pl.Utf8				
,'Total':pl.Float64
,'id':pl.Int64
,'name':pl.Utf8 
}

In [ ]:
# IMPORT DATA
data_08 = pl.read_csv(r'D:\Database\on_time_data\v2_on_time_performance_2023_08.csv',columns = fields)
data_09 = pl.read_csv(r'D:\Database\on_time_data\v2_on_time_performance_2023_09.csv',columns = fields)
data_10 = pl.read_csv(r'D:\Database\on_time_data\v2_on_time_performance_2023_10.csv',columns = fields)



In [ ]:
data = data_08.vstack(data_09)
data = data.vstack(data_10)

In [ ]:
del data_08,data_09,data_10
gc.collect()

# 2. Data Description
Assumption: Due to this only being an assessment, three months of data are used and only routes which are present and complete in all three months are included.

For the explorator analysis we will use 3 months of bus performance data download from Winnipeg's open data portal. This data set was supplemented with open weather data, stationary traffic count stations, neighbourhood shape files. links to the data are listed below


[Traffic Counts](https://data.winnipeg.ca/Transportation-Planning-Traffic-Management/Permanent-Count-Station-Traffic-Counts/46sc-6jrs/about_data)

[Neighbourhood](https://data.winnipeg.ca/City-Planning/Neighbourhood/fen6-iygi)

[Weather](https://www.visualcrossing.com/weather/weather-data-services/winnipeg/metric/last15days)

The data is stored in monthly files which contains information on departure times from bus stops for individual busses operating in the city. The column of interest in this dataset is the deviations columns, it represents the delayed time in seconds at a given stop. With negative values representing departure times that are behind schedule and positive values represent ahead of schedule. For This analysis we are using 3 months of data which results in roughly 12 million observations by 13 columns.

In [ ]:
print('Observations for August 2023 {}'.format(data_08.shape))
print('Observations for September 2023 {}'.format(data_09.shape))
print('Observations for October 2023 {}'.format(data_10.shape))
print('Observations Total {}'.format(data.shape))


# 3. Exploratory Analysis

In [ ]:
data.select(pl.selectors.by_dtype(pl.Float64, pl.Int64)).describe()

From the above data description we can see that there seems to be some outliers and erronious data in the deviation's columns. The min and max translate into a bus that is 5 hours late. This seems like an impossibility and should be investigated, but for the sake of this assessement outliers will be removed from the dataset.

In [ ]:
data = data.with_columns(
    pl.col('date_key').str.to_datetime("%Y-%m-%d %H:%M:%S")
)

In [ ]:
data = data.with_columns(
    pl.col('Day Type').map_elements(lambda x: 0 if x=="Weekday" else 1 ).alias('weekend')
)

In [ ]:
data = data.with_columns(
    pl.col('date_key').dt.month().alias('month'),
    pl.col('date_key').dt.day().alias('day'),
    pl.col('date_key').dt.hour().alias('hour'),
    pl.col('date_key').dt.weekday().alias('day_of_week')
)

## 3.1 Trend of Avg Delay Times For The City of Winnipeg

In [ ]:
data.head()

In [ ]:
temp_data = data.group_by([pl.col('month'),pl.col('day')]).agg(pl.col('Deviation').mean()).sort(['month','day']).with_columns(
    pl.date(pl.lit('2023'),pl.col('month'),pl.col('day')).alias('date')
).drop(['month','day']).to_pandas()

sns.lineplot(data=temp_data
             ,x = 'date'
             ,y = 'Deviation')
x = range(0, temp_data.shape[0])
z = np.polyfit(x, temp_data.Deviation, 1)
p = np.poly1d(z)
plt.plot(temp_data.date, p(x), c="b", ls=":")

del temp_data

### Observations
- It should be reconized that this small sample size will ignore any yearly effects the weather and construction patterns have on the delay of busses, but from first inspection it looks like that the bus system is performing worse across the sample period. Average Delay times are getting worse for the City of Winnipeg.
- The variability in the series is also increasing, this could be due to the worsning weather or a multitude of other factors. Weather will be investigated further.
 

# 3.1.1 Trend by Route Name - 10 Best and 10 Worst

In [ ]:
temp_data = data.group_by([pl.col('Route Name'),pl.col('month'),pl.col('day')]).agg(pl.col('Deviation').mean()).sort(['month','day']).with_columns(
    pl.date(pl.lit('2023'),pl.col('month'),pl.col('day')).alias('date'))


In [ ]:
worst_list = temp_data.group_by('Route Name')\
    .agg(pl.col('Deviation').mean())\
    .sort('Deviation').with_row_count()\
    .filter(pl.col('row_nr')<=10)\
    .select('Route Name')

worst_temp = temp_data.filter(pl.col('Route Name').is_in(list(worst_list.to_pandas().values.reshape(-1))))
# fig = px.line(worst_temp, x='date',y='Deviation', color = 'Route Name')
# fig.show()

sns.lineplot(data=worst_temp
             ,x = 'date'
             ,y = 'Deviation',hue='Route Name')

del worst_list,worst_temp

In [ ]:
best_list = temp_data.group_by('Route Name')\
    .agg(pl.col('Deviation').mean())\
    .sort('Deviation', descending=True).with_row_count()\
    .filter(pl.col('row_nr')<=10)\
    .select('Route Name')

best_temp = temp_data.filter(pl.col('Route Name').is_in(list(best_list.to_pandas().values.reshape(-1))))

sns.lineplot(data=best_temp 
             ,x = 'date'
             ,y = 'Deviation',hue='Route Name')

del best_temp,best_list

In [ ]:
del temp_data

### Observations for 10 best and worst Routes

After zooming in on the data to ignore the clear outlier on Broadway-william, the performance for the 10 best routes on average are remaining relatively flat. The driving force in the fall off in performance seems to be coming from a proportion of the data. It might be worth examing which routes are driving the fall off in performance. Do to the time limitation of this assessment we will focus on specific area's and weather for driving performance.

Future work could examine what purportion of routes have a decreasing performance, this would help focus analysis on routes that require attention.

# 3.1.2 Trend by Area of the city

In [ ]:
temp_data = data.group_by([pl.col('name'),pl.col('month'),pl.col('day')]).agg(pl.col('Deviation').mean()).sort(['month','day']).with_columns(
    pl.date(pl.lit('2023'),pl.col('month'),pl.col('day')).alias('date'))


In [ ]:
best_list = temp_data.group_by('name')\
    .agg(pl.col('Deviation').mean())\
    .sort('Deviation', descending=True).with_row_count()\
    .filter(pl.col('row_nr')<=10)\
    .select('name')

best_temp = temp_data.filter(pl.col('name').is_in(list(best_list.to_pandas().values.reshape(-1))))

sns.lineplot(data=best_temp 
             ,x = 'date'
             ,y = 'Deviation',hue='name')

del best_temp,best_list

In [ ]:
worst_list = temp_data.group_by('name')\
    .agg(pl.col('Deviation').mean())\
    .sort('Deviation', descending=False).with_row_count()\
    .filter(pl.col('row_nr')<=10)\
    .select('name')

worst_temp = temp_data.filter(pl.col('name').is_in(list(worst_list.to_pandas().values.reshape(-1))))

sns.lineplot(data=worst_temp 
             ,x = 'date'
             ,y = 'Deviation',hue='name')

del worst_temp,worst_list

### Observations of 10 best and worst performing neighbourhoods

Again the pattern repeats, the best performing areas are relatively flat and the worst are decreasing in performance. Interestingly there seems to be stronger seasonality pattern in the worst performing neighbourhoods suggesting there is a temperal component to the performance of these neighbourhoods.

## 3.2 Trend of Avg Delay Times by Day of Week

Next we will examine to see if there are specific days of the week that on average perform worse then the others


## 3.2.1 Box whisker plot by day for the Interquartile range for all routes

In [ ]:
# Due to size of data the IQR was plotted to check for seasonality
df = data.filter((pl.col('Deviation') > -207) & (pl.col('Deviation') <28)).to_pandas()
# fig = px.violin(data, y="Deviation", x="day_of_week", box=True, points="all",
#           hover_data=data.columns)
fig = px.box(df,x="day_of_week" ,y="Deviation")
fig.show()
del df
# sns.boxplot(data=data.filter((pl.col('Deviation') > -207) & (pl.col('Deviation') <28)).to_pandas(), x="day_of_week" ,y="Deviation")

In [ ]:

df = data.group_by([pl.col('name'),pl.col('day_of_week')]).agg(pl.col('Deviation').mean()).sort('name','day_of_week')

worst_avg = df.group_by('name').agg(pl.col('Deviation').mean()).sort('Deviation',descending=False).with_row_count().filter(pl.col('row_nr')<50).select('name')


df2 = df.filter(pl.col('name').is_in(list(worst_avg.to_pandas().values.reshape(-1)))).to_pandas()

new_df = df2.pivot(index='name', columns='day_of_week')['Deviation']
sns.heatmap(new_df)
# fig = px.imshow(new_df, x=new_df.columns, y=new_df.index)
# fig.update_layout(width=1000,height=2000)
# fig.show()
del df2,df,worst_avg
gc.collect()

## 3.2.2 Bar chart of counts for extreme events
- We defined an extreme event by taking 1.5*IQR - 25%quantile
- in this case the cut off was roughly -550

In [ ]:
#extreme value occurence
df = data.filter((pl.col('Deviation') < -559.5))
df = df.group_by('day_of_week').count().sort('day_of_week')

sns.barplot(x='day_of_week', y='count', data = df.to_pandas())
# fig = px.bar(df, x='day_of_week', y='count')
# fig.show()
del df


In [ ]:
df = data.filter((pl.col('Deviation') < -559.5))
df = df.group_by([pl.col('name'),pl.col('day_of_week')]).count().sort('name','day_of_week')
worst_count = df.group_by('name').agg(pl.col('count').mean()).sort('count',descending=False).with_row_count().filter(pl.col('row_nr')<50).select('name')


## 3.2.3 Heatmap of extreme events by neighbourhood

In [ ]:

df2 = df.filter(pl.col('name').is_in(list(worst_count.to_pandas().values.reshape(-1)))).to_pandas()
new_df = df2.pivot(index='name', columns='day_of_week')['count']

sns.heatmap(new_df)
del df2,df,worst_count
gc.collect()

### Observations by Day of week

- To examine the seasonal effect of the day of the week has on deviation times the data was split in two sets. First we can see that there is no daily effect on the interquartile range of data. Deviations that are typical don't seem to be correlated temporally to the day of the week.
- If we examine the extreme negative values a pattern starts to show up. Extreme values were defined using the interquartile method.
- It's easy to see from the above bar chart that more extreme deviations are likely to occure during the week. Which makes sense due to work traffic.
- From the heat chart the same pattern shows up, except it's broken out by neighbourhood. The worst neighbourhood for extreme deviations is South Portage.

# 3.3 Delay times by time of day

In [ ]:
temp_data = data.group_by('hour').agg(pl.col('Deviation').mean()).sort('hour')
sns.barplot( x='hour', y='Deviation', data = temp_data.to_pandas(),color='b')

del temp_data
gc.collect()

In [ ]:
temp_data = data.group_by([pl.col('name'),pl.col('hour')]).agg(pl.col('Deviation').mean()).sort('name','hour')
worst_avg = temp_data.group_by('name').agg(pl.col('Deviation').mean()).sort('Deviation',descending=True).with_row_count().filter(pl.col('row_nr')<50).select('name')

df2 = temp_data.filter(pl.col('name').is_in(list(worst_avg.to_pandas().values.reshape(-1)))).to_pandas()
new_df = df2.pivot(index='name', columns='hour')['Deviation'].fillna(0)


In [ ]:
sns.heatmap(new_df)
del temp_data,worst_avg,df2,new_df
gc.collect()

In [ ]:
#fitlering data for extreme events
df = data.filter((pl.col('Deviation') < -559.5))
df = df.group_by([pl.col('name'),pl.col('hour')]).count().sort('name','hour')
worst_count = df.group_by('name').agg(pl.col('count').mean()).sort('count',descending=True).with_row_count().filter(pl.col('row_nr')<50).select('name')


df2 = df.filter(pl.col('name').is_in(list(worst_count.to_pandas().values.reshape(-1)))).to_pandas()
new_df = df2.pivot(index='name', columns='hour')['count'].fillna(0)

sns.heatmap(new_df)
del df2,df,worst_count
gc.collect()

### Observations for time of day

- the problems time of day are 17:00 and 8:00
- Avg performance is the lowestest at 17:00
- the count of extreme events also peaks at 17:00
- interestlying there are different neighbourhoods present in the count of extreme events. Suggesting that they perform better on avg the the worst 50, but are more prone to extreme delays.

## 3.3 Trend of Deviations by weather

In [ ]:
data.head()

In [ ]:
data = data.with_columns(
    (pl.col('preciptype')).fill_null('No_precip'),
    pl.col('preciptype').map_elements(lambda x: 0 if x=="No_precip" else 1).alias('precip_ind')
)

In [ ]:
data.group_by('precip_ind').agg(mean_dev = pl.col('Deviation').mean(),
                                median_dev = pl.col('Deviation').median(),
                                std_dev = pl.col('Deviation').std())

In [ ]:
df = data.filter((pl.col('Deviation') < -559.5))
df = df.group_by([pl.col('precip_ind')]).agg(c_rows = pl.col('precip_ind').count(),
                                             mean_dev = pl.col('Deviation').mean())

# worst_count = df.group_by('name').agg(pl.col('count').mean()).sort('count',descending=True).with_row_count().filter(pl.col('row_nr')<50).select('name')

### Observation for Weather Impact

From this simple analysis there only seems to be a minor impact to bus performance, reducing the average deviation from -139 to -169 for all areas

# 4. Conclusion

Based on the above analysis we can draw a couple of conclusion based on the limited sample we drew. It seems that the city of Winnipeg transit system is getting worse. It is unclear whether this is a true effect or a typical seasonal effect that happens every year. In order to answer that a larger sample would have to be taken spanning multiple years. 

The major drivers based on this analysis are:
- Time of day, which acts as a proxy for traffic in the city. With 5pm being the worst time across all areas.
- Day of week
- Neighbourhood

There seems to be a subset of neighbourhoods that are driving the reduction in performance across this time frame. The top 10 neighbourhoods performance remained relatively flat across the period, whereas the worste 10 had a substantial drop. Although we can't draw major conclusions from this it suggests there are neighbourhoods outperforming others and a closer analysis should be done to identify the key driving differences.

# 5. Next steps

Further work should be done into identifying a good source of traffic data. As this seems to be the major factor. We should also attemp to correlate active construction projects with bus routes. A another major area that could yield active identifying links in the bus data, if the bus infront of the current bus is delayed the current bus will most likely be delayed as well. These issue are pretty subtaintial and should be over come before a serious effort at modeling this problem takes place, but for the sake of this assessment a toy model will be built using a paired down version of the data set. 